In [6]:
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
from face_recognition import face_distance
from datetime import datetime
import numpy as np
import cv2

# libraries for python firebase interaction
import pyrebase
import json
from collections import OrderedDict
import collections
from google.cloud import storage
from firebase import firebase
from urllib import request
import confg

In [7]:
class program:
    def __init__(self,):
        self.embedder = load_model('/home/ketan/3.hackathons/facenet_keras.h5')
        self.detector = MTCNN()
        self.tolerance = 12.5
        self.wanted = []
        self.wanted_dict = {}

    # Variables for python firebase interaction
        self.config = {
          "apiKey": confg.apiKey,
          "authDomain": confg.authDomain,
          "databaseURL": confg.databaseURL,
          "storageBucket": confg.storageBucket,
          "serviceAccount": "admin-sdk.json"
        }
        self.firebase = firebase.FirebaseApplication(confg.databaseURL)
        self.client = storage.Client.from_service_account_json('demo.json')
        self.bucket = self.client.get_bucket(confg.storageBucket)
        self.firebase1 = pyrebase.initialize_app(self.config)
        self.db = self.firebase1.database()
        self.last_update = {}
    
    def stream_handler(self, message):
        action = message['event']
        if action == 'patch':
            Data = message['data']
            print(Data)
            status = Data['status']
            if status == 0:
                while not self.bucket.get_blob(Data['img']):
                    if self.bucket.get_blob(Data['img']):
                        break
                xblob = self.bucket.get_blob(Data['img'])
                url = xblob.public_url
                print("url: ", url)
                req = request.urlopen(url)
                arr = np.asarray(bytearray(req.read()), dtype = np.uint8)
                img = cv2.imdecode(arr,-1)
                img = cv2.resize(img, (160, 160))
                encoding = self.find_embeddings([img])
                self.wanted_dict[Data['id']] = encoding
                self.wanted.append(encoding)
            else:
                try:
                    address = message['path']
                    address = address.split('/')
                    pop_img = address[2]
                    list_pop = self.wanted_dict[pop_img]
                    self.wanted.remove(list_pop)
                    del self.wanted_dict[pop_img]
                except:
                    pass
        elif action == 'put' and message['path'] == "/":
            try:
                x = message["data"]
                keys = x.keys()
                for key in keys:
                    f_keys = message['data'][key].keys()
                    for f_key in f_keys:
                        inner_dict = message['data'][key][f_key]
                        status = inner_dict['status']
                        if status == 0:
                            blob = self.bucket.get_blob(inner_dict['img'])
                            url = blob.public_url
                            print("url: ", url)
                            req = request.urlopen(url)
                            arr = np.asarray(bytearray(req.read()), dtype = np.uint8)
                            img1 = cv2.imdecode(arr,-1)
                            img1 = cv2.resize(img1, (160, 160))
                            encoding1 = self.find_embeddings([img1])
                            self.wanted_dict[f_key] = encoding1
                            self.wanted.append(encoding1[0])
                        else:
                            #Do Nothing as cause is already solved
                            pass
            except:
                pass
        
    def detect_face(self, image):
        return [face["box"] for face in self.detector.detect_faces(image)]
    
    def face_embedding(self, image, coordinates):
        print("Calculating Embeddings...", end = '\t')
        embeddings = []
        if len(coordinates) == 0:
            print("no face detected!!!")
            return embeddings
        for [x, y, w, h] in coordinates:
            face = cv2.resize(image[y:y+h, x:x+w], (160, 160))
            mean, std = face.mean(), face.std()
            face = (face - mean) / std
            samples = np.expand_dims(face, axis=0)
            embeddings.append(self.embeddings.predict(samples))
        print("completed")
        return embeddings
    def compareface(self, embeddings):
        print("compairing...")
        distances = []
        print("wanted : ", self.wanted)
        for wanted_index, wanted in enumerate(self.wanted):
            for index, face in enumerate(embeddings):
                temp_dist = face_distance(face, wanted)
                print("compairing wanted"+str(wanted_index)+" and face"+str(index)+" and result:", temp_dist)
                if temp_dist <= self.tolerance:
                    print("!!!!!!!!!!!!!!!!!!!!match found!!!!!!!!!!!!!!!!!!!!")
                    self.wanted_found(wanted_index, "default camera!")
                distances.append(temp_dist)
        print("compairing completed")
        
    def wanted_found(self, img, camera): # index of person detected in wanted, name of camera
        now = datetime.now()
        now = now.strftime("%m/%d/%Y, %H:%M")
        key = list(self.wanted_dict.keys())[img]
        try:
            lu = self.last_update[key]
            if lu == now:
                pass
                #Do Nothing
            else:
                self.last_update[key] = now
                cam = camera
                previous_list = self.db.child('policestation').child(key[:3]).child(key).child('seen').get()
                previous_list = list(previous_list.val())
                previous_list.append("On "+now+" at "+cam)
                self.db.child('policestation').child(key[:3]).child(key).update({"seen":previous_list})
        except KeyError:
            cam = camera
            previous_list = self.db.child('policestation').child(key[:3]).child(key).child('seen').get()
            previous_list = list(previous_list.val())
            previous_list.append("On "+now+" at "+cam)
            self.db.child('policestation').child(key[:3]).child(key).update({"seen":previous_list})
            self.last_update[key] = now

    def update_wanted(self, image):
        print("updating Wanted List...")
        coordinates = self.find_faces(image)
        encoding = self.find_embeddings(image, coordinates)
        self.wanted.append(encoding[0])
        print("wanted updation completed")

In [8]:
d = program()

FileNotFoundError: [Errno 2] No such file or directory: 'demo.json'

In [ ]:
my_stream = d.db.child("policestation").stream(d.stream_handler)

In [ ]:
cam = cv2.VideoCapture(0)
if (!cam.isOpened()):
    print('camera not working!!!')
    cam.release()

while 1:
    ok, frame = cam.read()
    if ok:
        coordinates = d.find_faces(frame)
        encodings = d.find_embeddings(frame, coordinates)
        d.compare(encodings)
        cv2.imshow("live image", frame)
        for index, face in enumerate(faces):
            cv2.imshow("face "+str(index), face)
        k = cv2.waitKey(2)
        if k & 0xff == ord('q'):
            cv2.destroyAllWindows()
            cam.release()
            break
    else:
        print('camera not working')

In [ ]:
cam = cv2.VideoCapture(0)
detector = MTCNN()
while 1:
    ok, frame = cam.read()
    if ok:
        faces = detector.detect_faces(frame)
        print([face["box"] for face in faces])
        cv2.imshow("live feed", frame)
        k = cv2.waitKey(2)
        if k & 0xff == ord("q"):
            cv2.destroyAllWindows()
            cam.release()
            break